In [5]:
import cv2
from PIL import Image
# from pdf2image import convert_from_path

In [6]:
import zipfile
import os

# Get the uploaded zip file name
zip_file_name = "/content/2023-20240613T072343Z-001.zip"

# Extract the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/pdfs')

# List the contents of the extracted folder
extracted_folder_path = '/content/pdfs'
extracted_files = os.listdir(extracted_folder_path)
print("Extracted files and folders:", extracted_files)

# If there is a single folder within the extracted files, navigate to it
pdf_folder = os.path.join(extracted_folder_path, extracted_files[0])
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
print("PDF files:", pdf_files)



Extracted files and folders: ['2023']
PDF files: ['20230818175109700630.pdf', '20240311105345742144.pdf', '20230608103554115216.pdf', '20240308145330234134.pdf', '20240308145829679839.pdf', '20230620162233579421.pdf', '20240311110807312852.pdf', '20230403123339004612   2023 शरकुलर.pdf', '20230216134112536906.pdf', '20230608104030828818.pdf', '20240311105654870946.pdf', '202307271812041149circular24.pdf', '20240311105519509045.pdf', '20230327132153939511 cr.pdf', '2023020917573151915 cir.pdf', '20230710170831306523 cir.pdf', '20230809120652518926.pdf', '20230202174013331804.pdf', '20240311105930757047.pdf', '20240308145511982336.pdf', '20230527141309399015 cir.pdf', '20230411162001136613   2023.pdf', '20240308145424331035.pdf', '20240311105158971042.pdf', '20230818175031989129.pdf', '20230620162122072520.pdf', '20230809112846205725.pdf', '20240311110301135950.pdf', '20230620162308836122.pdf', '20230314175534332610.pdf', '20230818175201505231.pdf', '20230202173930064103.pdf', '2024030814

In [7]:
# Step 2: Install and Configure Tesseract for Hindi
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-hin
!pip install pytesseract
!pip install PyMuPDF
!pip install pdf2image
!pip install pytesseract pdf2image opencv-python-headless pyspellchecker


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 1s (268 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [8]:
import os
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path

In [9]:
import numpy as np
import cv2
import pandas as pd
import pytesseract

In [13]:
import cv2
import numpy as np
import pytesseract
from pdf2image import convert_from_path
import csv
import os
import re

def process_img(image):
    # Resize the image
    res = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Thresholding
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # Rotation correction
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = res.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(res, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

def clean_text(text):
    # Remove multiple spaces and new lines, and structure the text
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(\.\s+)', '.\n', text)
    return text.strip()


In [14]:
def ocr_pdf(pdf_path, languages):
    # Convert PDF to images
    images = convert_from_path(pdf_path)
    full_text = ""
    # Process each image
    for img in images:
        # Perform OCR with specified languages
        img = np.array(img)  # Convert PIL image to numpy array
        img = process_img(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        text = pytesseract.image_to_string(img, lang="+".join(languages))
        clean_text_str = clean_text(text)
        full_text += clean_text_str + "\n"
    return full_text

def generate_csv(pdf_folder, pdf_files, languages):
    with open('ocr_output.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['File Name', 'OCR Text']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i, file_name in enumerate(pdf_files):
            if i >= 5:  # Process only the first 5 PDFs
                break
            pdf_path = os.path.join(pdf_folder, file_name)
            ocr_text = ocr_pdf(pdf_path, languages)
            writer.writerow({'File Name': file_name, 'OCR Text': ocr_text})
    print("CSV file generated successfully.")

def print_csv_contents(csv_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            print(f"File Name: {row['File Name']}")
            print(f"OCR Text: {row['OCR Text']}\n")

languages = ['eng', 'hin']

generate_csv(pdf_folder, pdf_files, languages)
print_csv_contents('ocr_output.csv')

CSV file generated successfully.
File Name: 20230818175109700630.pdf
OCR Text: विषय: प्रिय महोदय/महोदया, आप अवगत है कि विभिन्न न्यायालयों द्वारा विचारण के दौरान साक्ष्य हेतु पुलिस अधिकारियों / कर्मचारियों / टँ विजय कमार, ५ ॥ 008 >> ES डीजी परिपत्र सं0 - 3० /2023 SS le nh) पुलिस महानिदेशक, CEE उत्तर प्रदेश | PS पुलिस मुख्यालय, गोमती नगर विस्तार, लखनऊ-226002 दिनांकः अगस्त /&,2023 क्रिमिनल मिस बेल अप्लीकेशन सं0-9126/2023 जितेन्द्र बनाम उ0प्र0 राज्य व अन्य में मा0उच्च न्यायालय इलाहाबाद द्वारा पारित आदेश दिनांकित 10.07.2023 के अनुपालन गें सम्मनु/वारण्ट आदि के तामीला हेतु अन्य साह्षियों को रागानु/वारण्ट के माध्यम से आहूत किया जाता है, इन eh its 80 467 । ; om 0४ 2018 सम्मन/वारण्ट के तामीला में विलम्ब होने के कारण न्यायालय में चल डीजी परिपत्र सं0-27/2018 दि0-06.06.2018 रही विचारण की प्रक्रिया बाधित होती है। विभिन्न न्यायालयों द्वारा निर्मत डीजी परिपत्र सं0-60/2015 दि0-19.08.2015 सम्मन/वारण्टों के प्रभावी तामीला हेतु पूर्व में इस मुख्यालय स्तर से पार्धाकित बॉक्स में अंकित विभिन्न परिपत्रों के म